# Parcial final: Daniel Meneses Rojas C.C. 1036671105

### Modelo QML Titanic Original

## Librerias

In [1]:
import math                                                 # Libreria para calculos matemáticos
import pandas as pd                                         # Libreria para manipular de datos
import pennylane as qml                                     # Libreria necesaria para computación cuántica
from pennylane import numpy as np                           # Versión diferenciable de numpy
from pennylane.optimize import AdamOptimizer                # Optimizador Adam para el entrenamiento 

# Libreria para dividir los datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split        
# Métricas de evaluación
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

## **I.** Preprocesamiento de los datos de entrada

In [2]:
df_train = pd.read_csv('train.csv')                      # Carga el dataset desde un archivo CSV
df_train['Pclass'] = df_train['Pclass'].astype(str)      # Convierte la columna 'Pclass' a tipo string

# Codificación one-hot para las variables categóricas
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)

# Llena los valores faltantes de 'Age' con la mediana
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

# Crea una nueva columna para identificar si es niño (<12 años)
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)

# Selecciona las variables para el modelo
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

## **II.** División de datos de entrada y datos de prueba

In [3]:
# División de datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived'])  # División estratificada

# Conversión a arrays de Pennylane (diferenciables)
X_train = np.array(X_train.values, requires_grad=False)

# Conversión de etiquetas {0,1} a {-1,1} (necesario para el clasificador cuántico)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)), requires_grad=False)


## **III.** Etapa del Feature Map

In [4]:
# Configuración del dispositivo cuántico con 4 qubits
num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits)         # Simulador de qubits

# Etapa del Feature Map
def statepreparation(x):
    qml.BasisEmbedding(x, wires=range(0, num_qubits))       # Codifica los datos clásicos como estados base


## **IV.** Etapa del Ansatz

In [5]:
def layer(W):
    # Aplica rotaciones arbitrarias en cada qubit
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    # Aplica compuertas CNOT para entrelazar los qubits
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

# Definición del circuito cuántico completo
@qml.qnode(dev, interface="autograd")
def circuit(weights, x):
    statepreparation(x)                         # Aplica el feature map (codificación)
    for W in weights:
        layer(W)                                # Aplica cada capa del ansatz
    return qml.expval(qml.PauliZ(0))            # Retorna la expectativa de PauliZ en el qubit 0

# Función del clasificador variacional
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias           # Predicción = salida del circuito + sesgo

## **V.** Etapa de funcion de Pérdida

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += (l - p) ** 2           # Error cuadrático por muestra
    return loss / len(labels)         # Promedio del error

## **VI.** Etapa de evaluación de la Función de Costo

In [7]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]     # Clasifica todos los datos
    return square_loss(Y, predictions)                                      # Calcula el costo

## **VII.** Etapa de Análisis de Precisión

In [8]:
def accuracy(labels, predictions):
    correct = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:               # Cuenta como correcta si predicción y etiqueta coinciden
            correct += 1
    return correct / len(labels)            # Porcentaje de aciertos

## **VIII.** Pesos iniciales definidos

In [9]:
# Semilla para reproducibilidad
np.random.seed(0)               

# Inicialización aleatoria de pesos
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)  

# Sesgo inicial
bias_init = np.array(0.0, requires_grad=True)  

## **IX.** Pesos finales entrenados

In [10]:
opt = AdamOptimizer(0.125)                          # Optimizador Adam con tasa de aprendizaje 0.125
num_it = 30                                         # Número de iteraciones de entrenamiento
batch_size = math.floor(len(X_train) / num_it)      # Tamaño de minibatch

weights = weights_init                                 # Copia de los pesos
bias = bias_init

# Ciclo de entrenamiento
for it in range(num_it):
    # Selección aleatoria de minibatch
    batch_index = np.random.randint(0, len(X_train), (batch_size,))
    X_batch = X_train[batch_index]
    Y_batch = Y_train[batch_index]

    # Paso de optimización
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Cálculo de precisión en todo el conjunto de entrenamiento
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_train]
    acc = accuracy(Y_train, predictions)

    # Imprime información del progreso
    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f}".format(
            it + 1, cost(weights, bias, X_train, Y_train), acc
        )
    )


Iter:     1 | Cost: 2.3009047 | Accuracy: 0.3657928
Iter:     2 | Cost: 2.0028599 | Accuracy: 0.3657928
Iter:     3 | Cost: 1.6788013 | Accuracy: 0.3657928
Iter:     4 | Cost: 1.4103476 | Accuracy: 0.3570537
Iter:     5 | Cost: 1.2370707 | Accuracy: 0.6167291
Iter:     6 | Cost: 1.1592230 | Accuracy: 0.6167291
Iter:     7 | Cost: 1.1155736 | Accuracy: 0.6167291
Iter:     8 | Cost: 1.0833781 | Accuracy: 0.6167291
Iter:     9 | Cost: 1.0364816 | Accuracy: 0.6167291
Iter:    10 | Cost: 0.9838059 | Accuracy: 0.6167291
Iter:    11 | Cost: 0.9350933 | Accuracy: 0.6167291
Iter:    12 | Cost: 0.9040926 | Accuracy: 0.6779026
Iter:    13 | Cost: 0.8772236 | Accuracy: 0.6766542
Iter:    14 | Cost: 0.8435462 | Accuracy: 0.7802747
Iter:    15 | Cost: 0.8127882 | Accuracy: 0.7802747
Iter:    16 | Cost: 0.7705518 | Accuracy: 0.7802747
Iter:    17 | Cost: 0.7427210 | Accuracy: 0.7802747
Iter:    18 | Cost: 0.7298644 | Accuracy: 0.7802747
Iter:    19 | Cost: 0.7286801 | Accuracy: 0.7802747
Iter:    20 

## **X.** Evaluación de desempeño del clasificador cuantico final

In [11]:
# Evaluación de desempeño del clasificador
X_test = np.array(X_test.values, requires_grad=False)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)), requires_grad=False)

# Realiza las predicciones con el modelo de entrenado
predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

# Se evalua el desempeño con las metricas de exactitud, precisión y recall
print("Accuracy Score:", accuracy_score(Y_test, predictions))
print("Precision Score:", precision_score(Y_test, predictions))
print("Recall Score:", recall_score(Y_test, predictions))
print("F1 Score (macro):", f1_score(Y_test, predictions, average='macro'))


Accuracy Score: 0.7888888888888889
Precision Score: 0.7666666666666667
Recall Score: 0.6571428571428571
F1 Score (macro): 0.7712374581939799
